In [126]:
from tkinter import *
from tkinter import scrolledtext

In [127]:
from os import listdir, path, getcwd, remove
from collections import defaultdict

In [128]:
from datetime import datetime
import time

In [129]:
from tqdm.notebook import tqdm

In [143]:
# запуск

MyApp().window.mainloop()

In [137]:
class MyApp:
    
    def __init__(self):
        
        self.book_content, self.book_content_lower = Reader().book_content 
                
        self.window = Tk()
        self.window.title("Поиск книг")
        
        self.label = Label(self.window, 
            text="Введите ключевые слова (можно без окончаний) или словосочетания через запятую:", 
                           width = 100, height = 2, font = 15)
        self.label_request_title = Label(self.window, text="", fg="blue", 
                           width = 80, height = 1, font = 15)
        self.label_request = Label(self.window, text="", 
                           width = 80, height = 6, font = 15)
        self.label_result_title = Label(self.window, text="", fg="blue",
                           width = 80, height = 2, font = 15)        
        self.text_area = scrolledtext.ScrolledText(self.window, 
                           width = 80, height = 30, font = 10)
        self.entry = Entry(self.window,
                           width = 60, font = 15)
        self.button = Button(
            self.window, text="Поиск", command = self.print_request)
        
        
        self.label.grid(
            row=0, column = 0, columnspan = 2)
        self.entry.grid(
            row=1, column = 0)
        self.button.grid(
            row=1, column = 1)        
        self.label_request_title.grid(
            row=2, column = 0)
        self.label_request.grid(
            row=3, column = 0)        
        self.label_result_title.grid(
            row=4, column = 0)        
        self.text_area.grid(
            row=5, column = 0, pady = 20, padx = 20)        
      
    
    def print_request(self):
        
        key_word_string = self.entry.get()
        
        self.entry.delete(0, 'end')
        self.text_area.delete(0.0, 'end')
        
        self.label_request_title['text'] = f"Поиск по запросу:"
        
        key_word_list = key_word_string.lower().split(", ")
        key_word_print = '\n'.join(key_word_list)
        self.label_request['text'] =  f"{key_word_print}" 
        
        # обработка текста вывода:    
        book_dict_find = Librarian(key_word_list, self.book_content_lower).file_search()
        book_count = len(book_dict_find)
        
        text_context = ""
        dict_out = defaultdict(str) 
        for book, index_list in book_dict_find.items():
            len_conten = len(self.book_content[book])
            for i in index_list:                
                if i > 50:
                    dict_out[book] += f"... {self.book_content[book][(i - 50):(i + 50)]} ... -+++- "
                elif len_conten - i < 101:
                    dict_out[book] += f"... {self.book_content[book][i:]} ... -+++- "
                else:
                    dict_out[book] += f"... {self.book_content[book][:(i + 100)]} ... -+++- "
            text_context += f"{book}\n{dict_out[book]}\n\n"
        
        # вывод результа:
        self.label_result_title['text'] = f"Результат поиска: {book_count}"        
        self.text_area.insert(INSERT, f"{text_context}")

In [132]:
class Reader:
    
    def __init__(self):
        abs_path = path.join(getcwd(), "books")
        self.book_list = listdir(abs_path)
        self.book_content = self.file_content(self.book_list)
        
        
    def file_content(self, book_list):
        content_dict = {}
        content_dict_lower = {}
        for file_name in tqdm(self.book_list):            
            content_dict[file_name], content_dict_lower[file_name] = self._file_read(file_name)
        return content_dict, content_dict_lower           
    
       
    def _file_read(self, file_name):
        abs_path = path.join(getcwd(), f"books\\{file_name}")
        try:
            with open(abs_path, "r") as file:
                content = file.read().replace("\n", " ")
        except UnicodeDecodeError:
            with open(abs_path, "r", encoding="UTF-8") as file:
                content = file.read().replace("\n", " ")
        content_lower = content.lower()
        return content, content_lower

In [133]:
class Librarian:
    
    def __init__(self, key_word_list, book_content):
        self.key_word_list = key_word_list
        self.book_content = book_content
    
    
    def file_search(self):        
        book_dict_find = {}
        for book, content in self.book_content.items():
            key_word_list_find = []
            index_list = []
            flag = True
            for word in self.key_word_list:
                if flag == True:
                    i = content.find(word)
                    if i != -1:
                        key_word_list_find.append(word)
                        index_list.append(i)
                    else:
                        flag = False
            if self.key_word_list == key_word_list_find:
                book_dict_find[book] = index_list
        return book_dict_find

In [144]:
# Librarian ---  тесты

#book_list = listdir(path="./books")
file_content_list = MyApp().book_content_lower

s1 = ["что"]
s2 = ["АГАФЬЯ"]
s3 = ["пальто","человек"]
s4 = ["Надя","кухарке","барышне","барыне"]
s5 = ["кухар","жить","Надя","кухарке","барышне","барыне", "решительно"]
s6 = ["что","кто","как","и", "человек", "барышне","барыне", "решительно", "Надя"]
s7 = ["и чем кончился роман Шамохина - мне неизвестно."]
s8 = ["что кухарке действительно лучше жить, нежели барыне или барышне."]
print(s1)
start_time = datetime.now()
Librarian(s1, file_content_list).file_search()
print(datetime.now() - start_time)

print(s2)
start_time = datetime.now()
Librarian(s2, file_content_list).file_search()
print(datetime.now() - start_time)

print(s3)
start_time = datetime.now()
Librarian(s3, file_content_list).file_search()
print(datetime.now() - start_time)

print(s4)
start_time = datetime.now()
Librarian(s4, file_content_list).file_search()
print(datetime.now() - start_time)

print(s5)
start_time = datetime.now()
Librarian(s5, file_content_list).file_search()
print(datetime.now() - start_time)

print(s6)
start_time = datetime.now()
Librarian(s6, file_content_list).file_search()
print(datetime.now() - start_time)

print(s7)
start_time = datetime.now()
Librarian(s6, file_content_list).file_search()
print(datetime.now() - start_time)

print(s8)
start_time = datetime.now()
Librarian(s6, file_content_list).file_search()
print(datetime.now() - start_time)


['что']
0:00:00.001000
['АГАФЬЯ']
0:00:00.038897
['пальто', 'человек']
0:00:00.054852
['Надя', 'кухарке', 'барышне', 'барыне']
0:00:00.057354
['кухар', 'жить', 'Надя', 'кухарке', 'барышне', 'барыне', 'решительно']
0:00:00.076308
['что', 'кто', 'как', 'и', 'человек', 'барышне', 'барыне', 'решительно', 'Надя']
0:00:00.120188
['и чем кончился роман Шамохина - мне неизвестно.']
0:00:00.109708
['что кухарке действительно лучше жить, нежели барыне или барышне.']
0:00:00.126663
